# Data Preprocessing:

Using NLTK

Steps: 
1. Import data
2. Extract features



In [90]:
import nltk
# import ssl

# try:
#     _create_unverified_https_context = ssl._create_unverified_context
# except AttributeError:
#     pass
# else:
#     ssl._create_default_https_context = _create_unverified_https_context
# nltk.download('punkt')
# nltk.download('stopwords')
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.util import bigrams
from nltk.util import ngrams
import string
import pandas as pd
import random
random.seed(32)

# close nltk download window to continue

## Import data

In [50]:
data = pd.read_csv('pandora_bigfive1000.csv')
print(data.iloc[1:10]['body'])

1    Multiverses, matrix theory, consciousness. Scr...
2                      Hope you've enjoyed the ride :D
3    Idk, in the AFC if i recall correctly since 20...
4    22 hours later and the music is still going[co...
5                                               M’Lord
6    Quantum^Quantum^^Quantum^^^Quantum^^^^Quantum^...
7    You're a guy, aren't you? I can definitely see...
8    5/5 Lovely mayor! Hosted a giveaway, was very ...
9    That's pretty subjective, but I'd say you're i...
Name: body, dtype: object


## Feature extraction

In [46]:
# def features(data):

# Total number of characters (including space)
data['char_count'] = data['body'].str.len()

# Total number of stopwords
data['stopwords'] = data['body'].apply(lambda x: len([x for x in x.split() if x in stopwordList]))

# Total number of punctuation or special characters
data['total_punc'] = data['body'].apply(lambda x: len([x for x in x.split() if x in string.punctuation]))
# should not be 0 in all cases

# Total number of numerics
data['total_num'] = data['body'].apply(lambda x: len([x for x in x.split() if x.isdigit()]))

# Total number of uppercase words
data['total_uppercase'] = data['body'].apply(lambda x: len([x for x in x.split() if x.isupper()]))

data.head()

,author,author_flair_text,body,downs,created_utc,subreddit_id,link_id,parent_id,score,controversiality,...,subreddit,ups,word_count,word_count_quoteless,lang,char_count,stopwords,total_punc,total_num,total_uppercase
0,Sabata11792,NaN,Not seeing any break or signal lights and no p...,NaN,1534890968,t5_3fqup,t3_995l9s,t1_e4lbrls,1.0,0,...,ATBGE,NaN,19,19,en,101,7,0,0,0
1,Swarels,INTP,"Multiverses, matrix theory, consciousness. Scr...",NaN,1499749893,t5_2qhvl,t3_6mjw62,t1_dk26jre,7.0,0,...,INTP,NaN,51,46,en,338,16,0,0,2
2,pearlz176,Manchester United,Hope you've enjoyed the ride :D,NaN,1485613795,t5_2qi58,t3_5qnd1v,t1_dd0mdqg,2.0,0,...,soccer,NaN,6,6,en,31,2,0,0,1
3,rainbowhotpocket,Colts,"Idk, in the AFC if i recall correctly since 20...",NaN,1466965660,t5_2qmg3,t3_4pypuh,t1_d4oulvo,11.0,0,...,nfl,11.0,62,61,en,309,27,0,3,2
4,amathyx,http://myanimelist.net/profile/amathy,22 hours later and the music is still going[co...,NaN,1495851189,t5_2qh22,t3_6ddiow,t3_6ddiow,6.0,0,...,anime,NaN,32,29,en,268,12,0,2,0


## Preprocessing

1. lower 
2. tokenize

In [96]:
comments = data.filter(['author', 'body'])
# comments.head()

for i in range(len(comments)):
    # lowbody = [comments.iloc[i,1].lower()
    words = word_tokenize(comments.iloc[i,1])
    lowbody = [word.lower() for word in words if word.isalpha()]
    comments.iloc[i,1] = lowbody
    
comments.head()

,author,body
0,Sabata11792,"[not, seeing, any, break, or, signal, lights, ..."
1,Swarels,"[multiverses, matrix, theory, consciousness, s..."
2,pearlz176,"[hope, you, enjoyed, the, ride, d]"
3,rainbowhotpocket,"[idk, in, the, afc, if, i, recall, correctly, ..."
4,amathyx,"[hours, later, and, the, music, is, still, goi..."


In [97]:
# remove stopwords
stopwordList = stopwords.words('english')

contentwords = []
for sentence in comments['body']:
    contentwords.append([word for word in sentence if (word not in stopwordList)])
comments['body'] = contentwords
print(contentwords[1])
print(comments['body'])

['multiverses', 'matrix', 'theory', 'consciousness', 'scrubbing', 'reactions', 'quarks', 'lhclight', 'depends', 'day', 'oh', 'got', 'little', 'carried', 'away', 'definitely', 'canyons', 'rock', 'formations', 'carbon', 'microtubes', 'pretty', 'awesome', 'side']
0      [seeing, break, signal, lights, plate, probabl...
1      [multiverses, matrix, theory, consciousness, s...
2                                  [hope, enjoyed, ride]
3      [idk, afc, recall, correctly, since, teams, ma...
4      [hours, later, music, still, going, context, h...
                             ...                        
995                                       [fair, enough]
996    [maybe, marvin, lewis, secret, insignias, carv...
997    [would, flash, kernel, using, stock, rom, typi...
998    [ever, one, first, round, games, last, second,...
999                                [obviously, unstable]
Name: body, Length: 1000, dtype: object


Stopword removal works but negations ('not') are removed. Is that really what I want??

In [105]:
# Porter Stemmer
ps = PorterStemmer()

stemcomments = []
for sentence in comments['body']:
    stemword = []
    for word in sentence:
        stemword.append(ps.stem(word))
    stemcomments.append(stemword)
comments['body'] = stemcomments
print(comments['body'])
print(comments.iloc[1]['body'])

0      [see, break, signal, light, plate, probabl, al...
1      [multivers, matrix, theori, conscious, scrub, ...
2                                    [hope, enjoy, ride]
3      [idk, afc, recal, correctli, sinc, team, make,...
4      [hour, later, music, still, go, context, http,...
                             ...                        
995                                       [fair, enough]
996    [mayb, marvin, lewi, secret, insignia, carv, s...
997    [would, flash, kernel, use, stock, rom, typic,...
998    [ever, one, first, round, game, last, second, ...
999                                   [obvious, unstabl]
Name: body, Length: 1000, dtype: object
['multivers', 'matrix', 'theori', 'conscious', 'scrub', 'reaction', 'quark', 'lhclight', 'depend', 'day', 'oh', 'got', 'littl', 'carri', 'away', 'definit', 'canyon', 'rock', 'format', 'carbon', 'microtub', 'pretti', 'awesom', 'side']


In [43]:
# bigrams and trigrams
bi = []
for i in range(len(comments)):
    bi.append(list(bigrams(comments.iloc[i,1])))
comments['bigrams'] = bi

tri = []
for i in range(len(comments)):
    tri.append(list(ngrams(comments.iloc[i,1],n=3)))
comments['trigrams'] = tri
  
comments.head()

,author,body,bigrams,trigrams
0,Sabata11792,"[not, seeing, any, break, or, signal, lights, ...","[(not, seeing), (seeing, any), (any, break), (...","[(not, seeing, any), (seeing, any, break), (an..."
1,Swarels,"[multiverses, matrix, theory, consciousness, s...","[(multiverses, matrix), (matrix, theory), (the...","[(multiverses, matrix, theory), (matrix, theor..."
2,pearlz176,"[hope, you, enjoyed, the, ride, d]","[(hope, you), (you, enjoyed), (enjoyed, the), ...","[(hope, you, enjoyed), (you, enjoyed, the), (e..."
3,rainbowhotpocket,"[idk, in, the, afc, if, i, recall, correctly, ...","[(idk, in), (in, the), (the, afc), (afc, if), ...","[(idk, in, the), (in, the, afc), (the, afc, if..."
4,amathyx,"[hours, later, and, the, music, is, still, goi...","[(hours, later), (later, and), (and, the), (th...","[(hours, later, and), (later, and, the), (and,..."


However, these ngrams are not tf-idf weighted
Use:
https://www.geeksforgeeks.org/tf-idf-for-bigrams-trigrams/